

# Optimización de  Mixtral 8x7B usando PEFT y QLoRA

Este notebook muestra un laboratorio de como es posible afinal [Mixtral 8x7B](https://huggingface.co/mistralai/Mixtral-8x7B-v0.1) Mixture of Experts (MoE).


Este experimento utiliza QLoRA, un método de afinamiento que combina cuantización y LoRA. El LLM es cargado en 4 bits utilizando bitsandbytes y utiliza LoRA para entrenar usando la biblioteca PEFT de Hugging Face 🤗.

### 1. Creación del ambiente Cloud con GPU

Este laboratorio fue relizado usando un GPU y el ambiente de desarrollo de [brev.dev](https://brev.dev). Siguiendo el enlanc del badge es posible crear una instancia utilizando este Notebook:

[![](https://uohmivykqgnnbiouffke.supabase.co/storage/v1/object/public/landingpage/brevdeploynavy.svg)](https://console.brev.dev/environment/new?instance=T4:g4dn.12xlarge&diskStorage=512&name=mixtral-finetune-own-data&file=https://github.com/brevdev/notebooks/raw/main/mixtral-finetune-own-data.ipynb&python=3.10&cuda=12.1.1)

Como ambiente de ejecución se utilizó:

4xT4 (as linked) con 16GB GPU por GPU was enough for me . (3.91 USD/hora)

El ambiente de desarrollo utilizó  **Python 3.10 and CUDA 12.1.1**.



#### Paquetes necesarios



In [1]:
!pip install --upgrade pip
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U datasets scipy ipywidgets matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 16.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.

#### Accelerator
Se instala [Accelerator](https://huggingface.co/docs/accelerate/package_reference/accelerator), quizás no es del todo necesario por las optmizaciones aplicadas por LoRA.

In [2]:
from accelerate import FullyShardedDataParallelPlugin, Accelerator
from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig

fsdp_plugin = FullyShardedDataParallelPlugin(
    state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),
    optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True, rank0_only=False),
)

accelerator = Accelerator(fsdp_plugin=fsdp_plugin)

#### Weights & Biases para monitoreo de métricas


In [3]:
!pip install -q wandb -U

import wandb, os
wandb.login()

wandb_project = "gbif-datasets-peft"
if len(wandb_project) > 0:
    os.environ["WANDB_PROJECT"] = wandb_project

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.4 MB/s eta 0:00:00


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


#### Carga de datasets

In [4]:
from datasets import load_dataset

train_dataset = load_dataset('text', data_files='train.txt', split='train')
eval_dataset = load_dataset('text', data_files='test.txt', split='train')

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

#### Formatting prompts
Then create a `formatting_func` to structure training examples as prompts.

In [5]:
def formatting_func(dataset_txt):
    text = f"### {dataset_txt}"
    return text

### 2. Carga del modelo base

Se definen las configuraciones de cuantización, del modelo. 4 bits (load_in_4bit=True) con doble cuantización (bnb_4bit_use_double_quant=True), tipo de cálculo en 4 bits (bnb_4bit_compute_dtype=torch.bfloat16), carga en 8 bits con procesamiento en punto flotante de 32 bits en la CPU (load_in_8bit_fp32_cpu_offload=True) y habilita la cuantización en punto flotante de 32 bits en la CPU (llm_int8_enable_fp32_cpu_offload=True).

In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "mistralai/Mixtral-8x7B-v0.1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    # load_in_8bit_fp32_cpu_offload=True,
    # llm_int8_enable_fp32_cpu_offload=True
)

model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config, device_map="auto")

ValueError: ignored

### 3. Tokenización

Se define el tokenizador utilizando padding a la izquierda, en teoría utiliza [menos memoria](https://ai.stackexchange.com/questions/41485/while-fine-tuning-a-decoder-only-llm-like-llama-on-chat-dataset-what-kind-of-pa).

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    padding_side="left",
    add_eos_token=True,
    add_bos_token=True,
)
tokenizer.pad_token = tokenizer.eos_token

def generate_and_tokenize_prompt(prompt):
    return tokenizer(formatting_func(prompt))

Se reformatea los datos para el formato de prompt

In [ ]:
tokenized_train_dataset = train_dataset.map(generate_and_tokenize_prompt)
tokenized_val_dataset = eval_dataset.map(generate_and_tokenize_prompt)

Se obtiene una distribución de la longitude de los datasets para definir un `max_length` apropiado.

In [ ]:
import matplotlib.pyplot as plt

def plot_data_lengths(tokenized_train_dataset, tokenized_val_dataset):
    lengths = [len(x['input_ids']) for x in tokenized_train_dataset]
    lengths += [len(x['input_ids']) for x in tokenized_val_dataset]
    print(len(lengths))

    # Plotting the histogram
    plt.figure(figsize=(10, 6))
    plt.hist(lengths, bins=20, alpha=0.7, color='blue')
    plt.xlabel('Length of input_ids')
    plt.ylabel('Frequency')
    plt.title('Distribution of Lengths of input_ids')
    plt.show()

plot_data_lengths(tokenized_train_dataset, tokenized_val_dataset)

Se procede a tokenizar con padding y máximo de tamaño (truncamiento).[self-supervised fine-tuning is](https://neptune.ai/blog/self-supervised-learning).

In [ ]:
max_length = 2056

def generate_and_tokenize_prompt2(prompt):
    result = tokenizer(
        formatting_func(prompt),
        truncation=True,
        max_length=max_length,
        padding="max_length",
    )
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
tokenized_train_dataset = train_dataset.map(generate_and_tokenize_prompt2)
tokenized_val_dataset = eval_dataset.map(generate_and_tokenize_prompt2)

Se verifica que `input_ids` contiene el padding con  `eos_token` (2) que hay un `eos_token`al final, y que el prompt inicia con `bos_token` (1).

In [ ]:
print(tokenized_train_dataset[1]['input_ids'])

Todos los input debe ser del mismo tamaño, `max_length`.

In [ ]:
plot_data_lengths(tokenized_train_dataset, tokenized_val_dataset)

### 4. Configuración de LoRA

Para iniciar el afinamiento del modelo se utiliza el método`prepare_model_for_kbit_training` de PEFT.

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

Se imprime las capas del modelo y LoRA se aplica a todas las capas lineales del modelo `q_proj`, `k_proj`, `v_proj`, `o_proj`, `w1`, `w2`, `w3`, y `lm_head`.

In [ ]:
print(model)

Se definen las opciones de configuración de LoRA, en donde: `r` es el rango de la matriz de baja dimensión utilizada en los adaptadores, lo que controla la cantidad de parámetros entrenados. Un rango más alto permitirá una mayor expresividad, pero también conlleva un compromiso en términos de recursos de cómputo. `alpha` es el factor de escala para los pesos aprendidos. La matriz de pesos se escala por alpha/r, por lo que un valor más alto para alpha asigna más peso a las activaciones de LoRA.

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "w1",
        "w2",
        "w3",
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

See how the model looks different now, with the LoRA adapters added:

In [ ]:
print(model)

### 5. Entrenamiento del modelo

Se entrena el modelo utilizando max_steps=200 (iteraciones), este debe actualizar de acuerdo con el análisis de sobre ajuste del modelo.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    padding_side="left",
    add_eos_token=True,
    add_bos_token=True,
)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
if torch.cuda.device_count() > 1: # Si hay más de un GPU
    model.is_parallelizable = True
    model.model_parallel = True

In [ ]:
import transformers
from datetime import datetime

project = "gbif-datasets-lora"
base_model_name = "mixtral"
run_name = base_model_name + "-" + project
output_dir = "./" + run_name

trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    args=transformers.TrainingArguments(
        output_dir=output_dir,
        warmup_steps=1,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        gradient_checkpointing=True,
        max_steps=200,
        learning_rate=2.5e-5,
        fp16=True,
        optim="paged_adamw_8bit",
        logging_steps=25,
        logging_dir="./logs",
        save_strategy="steps",
        save_steps=25,
        evaluation_strategy="steps",
        eval_steps=25,
        do_eval=True,
        report_to="wandb",
        run_name=f"{run_name}-{datetime.now().strftime('%Y-%m-%d-%H-%M')}"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False
trainer.train()

### 6. Probar el modelo entrenado

Es recomendable detener el proceso actual e iniciar de nuevo el ambiente.
Dado que PEFT sólo guarda el valor de los adaptadores QLoRA es necesario cargar el modelo base desde Huggingface Hub:

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "mistralai/Mixtral-8x7B-v0.1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    use_auth_token=True
)

tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True)

Ahora se carga el adaptador QLoRA desde el directorio de checkpoint:

In [ ]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "mixtral-journal-finetune/checkpoint-200")

Ahora se evalua el modelo usando el mismo `eval_prompt` y `model_input`.
Se define una penaldad de repetición de 1.15.

In [ ]:
eval_prompt = " Describe the GBIF dataset 'Checklist of Vermont Species': # "
model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

ft_model.eval()
with torch.no_grad():
    print(tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=150,
                                             repetition_penalty=1.15)[0],
                           skip_special_tokens=True))